In [1]:
import pandas as pd
import numpy as np
import duckdb
import os
import sys
import sys
sys.path.insert(0, os.path.abspath("../.."))
from utils import time, data_processor, constants

In [2]:
all_countries = set(constants.eu + constants.americas + constants.asia + constants.africa + constants.oceania)
eu = set(constants.eu)
balkans = set(constants.balkans)

In [13]:
INDUSTRY_CLASSIFICATION_PATH = "../../data_raw/firmographics/industry_classifications/*.parquet"
CONTACT_INFO_PATH = "../../data_raw/firmographics/contact_info/*.parquet"
LEGAL_INFO_PATTERN = "../../data_raw/firmographics/legal_info/*.parquet"
ALL_ADDRESSES = "../../data_raw/firmographics/all_addresses/*.parquet"


In [14]:
# def fetch_data(country_iso_code):
#     query = f"""
#     SELECT
#         *
#     FROM
#         '{CONTACT_INFO_PATH}' as nace
#     WHERE
#         nace."bvd_id_number" LIKE '{country_iso_code}%'

#     """
#     conn = duckdb.connect()
#     df = conn.execute(query).fetchdf()
    
#     return df

# # ita_industry_classification = fetch_data("IT")
# ita_contact_info = fetch_data("IT")
# # ita_legal_info = fetch_data("IT")

## NON-HISTORICAL DATA

In [17]:
def fetch_data_for_country(country_iso_code):
    query = f"""
        SELECT 
            nace.bvd_id_number,
            nace.nace_rev_2_core_code_4_digits_,
            contact.nuts2,
            contact.city_native_,
            legal_info.status,
            legal_info.type_of_entity,
            all_addresses.postcode,
            all_addresses.country_iso_code,
            all_addresses.city_native_,
            all_addresses.city,
            all_addresses.region_in_country,
        FROM
            '{INDUSTRY_CLASSIFICATION_PATH}' AS nace
        LEFT JOIN
            '{CONTACT_INFO_PATH}' AS contact
        ON
            nace.bvd_id_number = contact.bvd_id_number
        LEFT JOIN
            '{LEGAL_INFO_PATTERN}' AS legal_info
        ON
            nace.bvd_id_number = legal_info.bvd_id_number
        LEFT JOIN
            '{ALL_ADDRESSES}' AS all_addresses
        ON  
            all_addresses.bvd_id_number = nace.bvd_id_number
        WHERE
            nace.bvd_id_number IS NOT NULL
        AND 
            (nace.nace_rev_2_core_code_4_digits_ IS NOT NULL 
            OR contact.nuts2 IS NOT NULL 
            OR legal_info.status IS NOT NULL 
            OR legal_info.type_of_entity IS NOT NULL)
        AND
            contact.country_iso_code = '{country_iso_code}'
    """

    conn = duckdb.connect()
    df = conn.execute(query).fetchdf()
    
    return df



In [18]:
country = 'CN' 

al =  fetch_data_for_country(country)

In [20]:
al

,bvd_id_number,nace_rev_2_core_code_4_digits_,nuts2,city_native_,status,type_of_entity,postcode,country_iso_code,city_native__1,city,region_in_country
0,CN9384786159,7112,None,北京,Dissolved,Corporate,000000,CN,北京,Beijing,North China|Beijing
1,CN9384780951,4643,None,None,Active,Corporate,226001,CN,None,None,North China|Beijing
2,CN9384780991,1396,None,None,Active,Financial company,226008,CN,None,None,North China|Beijing
3,CN9384786124,4743,None,北京市,Dissolved,Corporate,100080,CN,北京市,Beijing,North China|Beijing
4,CN9384784238,6831,None,北京,Dissolved,Corporate,000000,CN,北京,Beijing,North China|Beijing
...,...,...,...,...,...,...,...,...,...,...,...
57864380,CN9360840278,None,None,北京市,Status unknown,Corporate,102445,CN,北京市,Beijing,None
57864381,CN9366872883,None,None,张家口市,Active,Corporate,000000,CN,张家口市,Zhangjiakou,North China|Hebei
57864382,CN9360098224,None,None,阜新,Active,Corporate,123125,CN,阜新,Fuxin,North East China|Liaoning
57864383,CN9360156656,None,None,None,Active,Corporate,000000,CN,None,None,East China|Fujian


In [19]:
output_path = "..\\..\\data_processed\\firmographics_processed\\"

In [21]:
# output_path = "Z:\\dati_moody\\data_processed\\firmographics_processed\\firmographics_CN.parquet"

for country in ['CN']:
    df = fetch_data_for_country(country)
    file_path = f"{output_path}firmographics_{country}.parquet"

    # Check if the file exists and remove it if necessary
    if os.path.exists(file_path):
        os.remove(file_path)

    # Write the DataFrame to parquet
    df.to_parquet(file_path)
    print(f"{country}")


CN


In [14]:
# TEMP_TABLE_KEY_FINANCIALS = "..\\..\\data_raw\\key_financials\\key_financials_eur\\*.parquet"

# def fetch_data_for_country_historical(country_iso_code):
#     query = f"""
#     SELECT 
#         number_of_employees, 
#         closing_date,
#         bvd_id_number,
#         total_assets,
#         operating_revenue_turnover_,
#         EXTRACT(YEAR FROM closing_date) AS year  
#     FROM 
#         '{TEMP_TABLE_KEY_FINANCIALS}' 
#     WHERE
#         bvd_id_number LIKE '{country_iso_code}%'
#         """
    
#     # Connect to DuckDB and execute the query   
#     conn = duckdb.connect()
#     df = conn.execute(query).fetchdf()
#     return df

In [ ]:
# ## Test
# # country = "IT"
# OUTPUT_PATH = "..\\..\\data_processed\\key_financials_processed\\"

# for country in all_countries:
#     df = fetch_data_for_country_historical(country)
#     os.makedirs(OUTPUT_PATH, exist_ok=True)
#     df.to_parquet(f"{OUTPUT_PATH}\\key_financials_processed_{country}.parquet")
#     print(f"{country}")
